In [2]:
import pandas as pd
import unicodedata
import spacy
from langdetect import detect
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
import spacy.util
import matplotlib.pyplot as plt
from collections import Counter

## 1. Limpieza de las csv jobs

In [3]:
jobs_andres = pd.read_csv("C:/Users/brian.amaya/Documents/2.Proyectos/jobs_bioeconomy/raw_data/TestGreenScore.csv")
jobs_andres = jobs_andres [["ID_file", "title_kw","title_kw_des"]]

jobs_linkedin = pd.read_csv("C:/Users/brian.amaya/Documents/2.Proyectos/Bedex/dash_news/raw_data/jobs/job_details.csv")
jobs_linkedin = jobs_linkedin [["keyword","current_jobid", "job_title","description_job"]]
jobs_linkedin = jobs_linkedin.rename(columns={
    "current_jobid": "ID_file",
    "job_title": "title_kw",
    "description_job": "title_kw_des"
})

# Ejemplo de uso

#jobs_df = jobs_andres
#jobs_df = jobs_linkedin


In [4]:
def remove_accents(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    return ''.join([char for char in nfkd_form if not unicodedata.combining(char)])  # Normalizar el texto eliminando tildes


jobs_linkedin['title_kw_des'] = jobs_linkedin['title_kw_des'].apply(remove_accents)

# Cargar los modelos de spaCy en español y en inglés
nlp_es = spacy.load("es_core_news_md")
nlp_en = spacy.load("en_core_web_sm")

# Cargar los archivos CSV con las palabras clave y sectores en español e inglés
df_keywords_es = pd.read_csv('tidy_data/in_es_keywords.csv')
df_keywords_en = pd.read_csv('tidy_data/in_en_keywords.csv')

# Limpiar y preparar los datos del DataFrame de palabras clave en español
df_keywords_es['sectors'] = df_keywords_es['sectors'].fillna('Unknown')
df_keywords_es['sectors'] = df_keywords_es['sectors'].astype(str)

# Organizar las palabras clave por sector en español
keywords_by_sector_es = {}
for index, row in df_keywords_es.iterrows():
    sector = row['sectors']
    keyword = remove_accents(row['keyword_es'])  # Normalizar palabras clave eliminando tildes
    if sector not in keywords_by_sector_es:
        keywords_by_sector_es[sector] = []
    keywords_by_sector_es[sector].append(keyword)

# Limpiar y preparar los datos del DataFrame de palabras clave en inglés
df_keywords_en['sectors'] = df_keywords_en['sectors'].fillna('Unknown')
df_keywords_en['sectors'] = df_keywords_en['sectors'].astype(str)

# Organizar las palabras clave por sector en inglés
keywords_by_sector_en = {}
for index, row in df_keywords_en.iterrows():
    sector = row['sectors']
    keyword = remove_accents(row['keyword_en'])  # Normalizar palabras clave eliminando tildes
    if sector not in keywords_by_sector_en:
        keywords_by_sector_en[sector] = []
    keywords_by_sector_en[sector].append(keyword)

def process_jobs(jobs_df, language='es'):
    # Identificar el idioma de cada descripción
    jobs_df['language'] = jobs_df['title_kw_des'].apply(lambda x: detect(x))

    # Filtrar los DataFrames según el idioma
    df_es = jobs_df[jobs_df['language'] == 'es']
    df_en = jobs_df[jobs_df['language'] == 'en']

    if language == 'es':
        matcher = PhraseMatcher(nlp_es.vocab, attr="LEMMA")
        for sector, keywords in keywords_by_sector_es.items():
            patterns = [nlp_es(text.lower()) for text in keywords]
            matcher.add(sector.upper(), None, *patterns)
        
        nlp = nlp_es
        df_to_process = df_es
        keywords_by_sector = keywords_by_sector_es
    else:
        matcher = PhraseMatcher(nlp_en.vocab, attr="LEMMA")
        for sector, keywords in keywords_by_sector_en.items():
            patterns = [nlp_en(text.lower()) for text in keywords]
            matcher.add(sector.upper(), None, *patterns)
        
        nlp = nlp_en
        df_to_process = df_en
        keywords_by_sector = keywords_by_sector_en

    processed_data = []

    for index, row in df_to_process.iterrows():
        description = row['title_kw_des']
        
        # Procesar el texto
        doc = nlp(description.lower())

        # Encontrar las coincidencias y etiquetarlas
        matches = matcher(doc)
        spans = []
        for match_id, start, end in matches:
            if not any(span.start <= start < span.end or span.start < end <= span.end for span in spans):
                spans.append(Span(doc, start, end, label=nlp.vocab.strings[match_id]))

        # Eliminar duplicados y solapamientos potenciales
        spans = spacy.util.filter_spans(spans)
        doc.ents = spans

        # Extraer datos para el DataFrame
        data = {sector.upper(): [] for sector in keywords_by_sector}
        for ent in doc.ents:
            if ent.label_ not in data:
                data[ent.label_] = []
            data[ent.label_].append(ent.text)

        # Convertir listas de palabras en cadenas separadas por comas
        for sector in data:
            data[sector] = ', '.join(set(data[sector]))

        # Agregar los datos del DataFrame original
        data['ID_file'] = row['ID_file']
        data['title_kw'] = row['title_kw']
        data['Processed_Text'] = description
        processed_data.append(data)

    # Crear DataFrame con la información recopilada
    df_final = pd.DataFrame(processed_data)
    # Reordenar columnas
    df_final = df_final[['ID_file', 'title_kw', 'Processed_Text'] + [col for col in df_final.columns if col not in ['ID_file', 'title_kw', 'Processed_Text']]]

    # Crear columnas de conteo
    for sector in keywords_by_sector:
        df_final[sector.upper() + '_COUNT'] = df_final[sector.upper()].apply(lambda x: len(x.split(', ')) if x else 0)

    # Guardar el DataFrame en un archivo CSV
    df_final.to_csv(f'processed_jobs_{language}.csv', index=False)

    return df_final

# Procesar y guardar DataFrames para español e inglés
df_es = process_jobs(jobs_linkedin, language='es')
df_en = process_jobs(jobs_linkedin, language='en')

In [6]:
import pandas as pd
import spacy
import unicodedata
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from langdetect import detect

# Función para eliminar tildes
def remove_accents(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    return ''.join([char for char in nfkd_form if not unicodedata.combining(char)])

jobs_linkedin['title_kw_des'] = jobs_linkedin['title_kw_des'].apply(remove_accents)

# Cargar los modelos de spaCy en español, inglés y portugués
nlp_es = spacy.load("es_core_news_md")
nlp_en = spacy.load("en_core_web_sm")
nlp_pt = spacy.load("pt_core_news_md")

# Cargar los archivos CSV con las palabras clave y sectores en español, inglés y portugués
df_keywords_es = pd.read_csv('tidy_data/in_es_keywords.csv')
df_keywords_en = pd.read_csv('tidy_data/in_en_keywords.csv')
df_keywords_pt = pd.read_csv('tidy_data/in_pt_keywords.csv')

# Limpiar y preparar los datos del DataFrame de palabras clave en español
df_keywords_es['sectors'] = df_keywords_es['sectors'].fillna('Unknown')
df_keywords_es['sectors'] = df_keywords_es['sectors'].astype(str)

# Organizar las palabras clave por sector en español
keywords_by_sector_es = {}
for index, row in df_keywords_es.iterrows():
    sector = row['sectors']
    keyword = remove_accents(row['keyword_es'])  # Normalizar palabras clave eliminando tildes
    if sector not in keywords_by_sector_es:
        keywords_by_sector_es[sector] = []
    keywords_by_sector_es[sector].append(keyword)

# Limpiar y preparar los datos del DataFrame de palabras clave en inglés
df_keywords_en['sectors'] = df_keywords_en['sectors'].fillna('Unknown')
df_keywords_en['sectors'] = df_keywords_en['sectors'].astype(str)

# Organizar las palabras clave por sector en inglés
keywords_by_sector_en = {}
for index, row in df_keywords_en.iterrows():
    sector = row['sectors']
    keyword = remove_accents(row['keyword_en'])  # Normalizar palabras clave eliminando tildes
    if sector not in keywords_by_sector_en:
        keywords_by_sector_en[sector] = []
    keywords_by_sector_en[sector].append(keyword)

# Limpiar y preparar los datos del DataFrame de palabras clave en portugués
df_keywords_pt['sectors'] = df_keywords_pt['sectors'].fillna('Unknown')
df_keywords_pt['sectors'] = df_keywords_pt['sectors'].astype(str)

# Organizar las palabras clave por sector en portugués
keywords_by_sector_pt = {}
for index, row in df_keywords_pt.iterrows():
    sector = row['sectors']
    keyword = remove_accents(row['keyword_pt'])  # Normalizar palabras clave eliminando tildes
    if sector not in keywords_by_sector_pt:
        keywords_by_sector_pt[sector] = []
    keywords_by_sector_pt[sector].append(keyword)

def process_jobs(jobs_df, language='es'):
    # Identificar el idioma de cada descripción
    jobs_df['language'] = jobs_df['title_kw_des'].apply(lambda x: detect(x))

    # Filtrar los DataFrames según el idioma
    df_es = jobs_df[jobs_df['language'] == 'es']
    df_en = jobs_df[jobs_df['language'] == 'en']
    df_pt = jobs_df[jobs_df['language'] == 'pt']

    if language == 'es':
        matcher = PhraseMatcher(nlp_es.vocab, attr="LEMMA")
        for sector, keywords in keywords_by_sector_es.items():
            patterns = [nlp_es(text.lower()) for text in keywords]
            matcher.add(sector.upper(), None, *patterns)
        
        nlp = nlp_es
        df_to_process = df_es
        keywords_by_sector = keywords_by_sector_es
    elif language == 'en':
        matcher = PhraseMatcher(nlp_en.vocab, attr="LEMMA")
        for sector, keywords in keywords_by_sector_en.items():
            patterns = [nlp_en(text.lower()) for text in keywords]
            matcher.add(sector.upper(), None, *patterns)
        
        nlp = nlp_en
        df_to_process = df_en
        keywords_by_sector = keywords_by_sector_en
    elif language == 'pt':
        matcher = PhraseMatcher(nlp_pt.vocab, attr="LEMMA")
        for sector, keywords in keywords_by_sector_pt.items():
            patterns = [nlp_pt(text.lower()) for text in keywords]
            matcher.add(sector.upper(), None, *patterns)
        
        nlp = nlp_pt
        df_to_process = df_pt
        keywords_by_sector = keywords_by_sector_pt

    processed_data = []

    for index, row in df_to_process.iterrows():
        description = row['title_kw_des']
        
        # Procesar el texto
        doc = nlp(description.lower())

        # Encontrar las coincidencias y etiquetarlas
        matches = matcher(doc)
        spans = []
        for match_id, start, end in matches:
            if not any(span.start <= start < span.end or span.start < end <= span.end for span in spans):
                spans.append(Span(doc, start, end, label=nlp.vocab.strings[match_id]))

        # Eliminar duplicados y solapamientos potenciales
        spans = spacy.util.filter_spans(spans)
        doc.ents = spans

        # Extraer datos para el DataFrame
        data = {sector.upper(): [] for sector in keywords_by_sector}
        for ent in doc.ents:
            if ent.label_ not in data:
                data[ent.label_] = []
            data[ent.label_].append(ent.text)

        # Convertir listas de palabras en cadenas separadas por comas
        for sector in data:
            data[sector] = ', '.join(set(data[sector]))

        # Agregar los datos del DataFrame original
        data['ID_file'] = row['ID_file']
        data['title_kw'] = row['title_kw']
        data['Processed_Text'] = description
        processed_data.append(data)

    # Crear DataFrame con la información recopilada
    df_final = pd.DataFrame(processed_data)
    # Reordenar columnas
    df_final = df_final[['ID_file', 'title_kw', 'Processed_Text'] + [col for col in df_final.columns if col not in ['ID_file', 'title_kw', 'Processed_Text']]]

    # Crear columnas de conteo
    for sector in keywords_by_sector:
        df_final[sector.upper() + '_COUNT'] = df_final[sector.upper()].apply(lambda x: len(x.split(', ')) if x else 0)

    # Guardar el DataFrame en un archivo CSV
    df_final.to_csv(f'processed_jobs_{language}.csv', index=False)

    return df_final

# Procesar y guardar DataFrames para español, inglés y portugués
df_es = process_jobs(jobs_linkedin, language='es')
df_en = process_jobs(jobs_linkedin, language='en')
df_pt = process_jobs(jobs_linkedin, language='pt')

In [4]:
def count_words(df, column):
    all_words = ', '.join(df[column].dropna()).split(', ')
    all_words = [word.strip() for word in all_words if word.strip()]  # Eliminar espacios adicionales y vacíos
    return Counter(all_words)

# Categorías a considerar
categories = ['TRANSVERSAL', 'AGRO', 'PISCÍCOLA', 'SALUD', 'BIOTECNOLOGÍA', 'RESIDUOS', 'FORESTAL', 'ENERGÍA', 'TURISMO']

# Lista para almacenar los datos procesados
processed_data = []

# Procesar los datos para cada categoría en español
for category in categories:
    if category in df_es.columns:
        word_counts = count_words(df_es, category)
        for word, count in word_counts.items():
            processed_data.append({'Categoria': category, 'Palabra': word, 'Conteo': count, 'Language': 'es'})

# Procesar los datos para cada categoría en inglés
for category in categories:
    if category in df_en.columns:
        word_counts = count_words(df_en, category)
        for word, count in word_counts.items():
            processed_data.append({'Categoria': category, 'Palabra': word, 'Conteo': count, 'Language': 'en'})

# Crear DataFrame con la información recopilada
df_count_es_en = pd.DataFrame(processed_data)

In [5]:
df_count_es_en.to_csv('df_count_es_en.csv', index=False, encoding="utf-8")

In [7]:
# Función para eliminar tildes
def remove_accents(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    return ''.join([char for char in nfkd_form if not unicodedata.combining(char)])

In [6]:

# Normalizar el texto eliminando tildes
jobs_df['title_kw_des'] = jobs_df['title_kw_des'].apply(remove_accents)

# Cargar los modelos de spaCy en español y en inglés
nlp_es = spacy.load("es_core_news_md")
nlp_en = spacy.load("en_core_web_sm")

# Cargar el archivo CSV con las palabras clave y sectores
df_keywords = pd.read_csv('tidy_data/in_es_keywords.csv')

# Limpiar y preparar los datos del DataFrame de palabras clave
df_keywords['sectors'] = df_keywords['sectors'].fillna('Unknown')
df_keywords['sectors'] = df_keywords['sectors'].astype(str)

# Organizar las palabras clave por sector
keywords_by_sector = {}
for index, row in df_keywords.iterrows():
    sector = row['sectors']
    keyword = remove_accents(row['keyword_es'])  # Normalizar palabras clave eliminando tildes
    if sector not in keywords_by_sector:
        keywords_by_sector[sector] = []
    keywords_by_sector[sector].append(keyword)

# Inicializar el PhraseMatcher y añadir patrones
matcher = PhraseMatcher(nlp_es.vocab, attr="LEMMA")
for sector, keywords in keywords_by_sector.items():
    patterns = [nlp_es(text.lower()) for text in keywords]
    matcher.add(sector.upper(), None, *patterns)

# Lista para almacenar los datos procesados
processed_data = []

# Iterar sobre cada descripción en el archivo CSV
for index, row in jobs_df.iterrows():
    description = row['title_kw_des']
    
    # Procesar el texto
    doc = nlp_es(description.lower())

    # Encontrar las coincidencias y etiquetarlas
    matches = matcher(doc)
    spans = []
    for match_id, start, end in matches:
        if not any(span.start <= start < span.end or span.start < end <= span.end for span in spans):
            spans.append(Span(doc, start, end, label=nlp_es.vocab.strings[match_id]))

    # Eliminar duplicados y solapamientos potenciales
    spans = spacy.util.filter_spans(spans)
    doc.ents = spans

    # Extraer datos para el DataFrame
    data = {sector.upper(): [] for sector in keywords_by_sector}
    for ent in doc.ents:
        if ent.label_ not in data:
            data[ent.label_] = []
        data[ent.label_].append(ent.text)

    # Convertir listas de palabras en cadenas separadas por comas
    for sector in data:
        data[sector] = ', '.join(set(data[sector]))

    # Agregar los datos del DataFrame original
    data['ID_file'] = row['ID_file']
    data['title_kw'] = row['title_kw']
    data['Processed_Text'] = description
    processed_data.append(data)

# Crear DataFrame con la información recopilada
df_final = pd.DataFrame(processed_data)
# Reordenar columnas
df_final = df_final[['ID_file', 'title_kw', 'Processed_Text'] + [col for col in df_final.columns if col not in ['ID_file', 'title_kw', 'Processed_Text']]]

# Crear columnas de conteo
for sector in keywords_by_sector:
    df_final[sector.upper() + '_COUNT'] = df_final[sector.upper()].apply(lambda x: len(x.split(', ')) if x else 0)

# Imprimir los resultados
print("Tabla de Entidades y Conteos:")
print(df_final.head())


NameError: name 'jobs_df' is not defined

In [ ]:
# Exportar a CSV
df_final.to_csv('jobs_andres.csv', index=False, encoding="utf-8")

In [9]:
# Exportar a CSV
df_final.to_csv('raw_data/jobs_linkedin.csv', index=False, encoding="utf-8")